In [27]:
# used quant_trading_projects/macd_strategy/ema_strategy.ipynb as reference
from splinter import Browser
from bs4 import BeautifulSoup 
from selenium import webdriver
import pprint


In [6]:
# Visit the website
# Initialize the Firefox WebDriver with Geckodriver

driver = webdriver.Firefox()
url = f'https://www.slickcharts.com/sp500'
driver.get(url)

# Create a BeautifulSoup parser to parse the HTML code
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')

# pulling SP500 tickers and putting them onto a list
table = soup.find('div', class_='table-responsive')
rows = table.find_all('tr')
rows = rows[1:]
rows

tickers = []

for row in rows:
    symbol = row.find_all('td')
    ticker = symbol[2].text
    
    tickers.append(ticker)

# Quit your browsing session
driver.quit()

# reformat ticker symbols to match Yfinance ticker formatting
ticker_symbols = [ticker.replace('.','-') if '.' in ticker else ticker for ticker in tickers]

In [16]:
ticker_symbols

['AAPL',
 'MSFT',
 'NVDA',
 'AMZN',
 'META',
 'TSLA',
 'GOOGL',
 'AVGO',
 'GOOG',
 'BRK-B',
 'JPM',
 'LLY',
 'V',
 'XOM',
 'UNH',
 'COST',
 'MA',
 'HD',
 'WMT',
 'PG',
 'NFLX',
 'JNJ',
 'CRM',
 'BAC',
 'ABBV',
 'ORCL',
 'CVX',
 'MRK',
 'KO',
 'WFC',
 'CSCO',
 'NOW',
 'ACN',
 'PEP',
 'MCD',
 'IBM',
 'LIN',
 'ADBE',
 'AMD',
 'DIS',
 'TMO',
 'ABT',
 'PM',
 'ISRG',
 'GS',
 'CAT',
 'INTU',
 'GE',
 'VZ',
 'QCOM',
 'BKNG',
 'TXN',
 'AXP',
 'T',
 'SPGI',
 'MS',
 'RTX',
 'CMCSA',
 'PLTR',
 'NEE',
 'DHR',
 'LOW',
 'HON',
 'PGR',
 'BLK',
 'AMGN',
 'PFE',
 'UNP',
 'ETN',
 'TJX',
 'AMAT',
 'BX',
 'C',
 'BSX',
 'COP',
 'SYK',
 'PANW',
 'UBER',
 'BA',
 'ADP',
 'VRTX',
 'SCHW',
 'FI',
 'ANET',
 'GILD',
 'MU',
 'TMUS',
 'BMY',
 'DE',
 'SBUX',
 'ADI',
 'MDT',
 'MMC',
 'LMT',
 'KKR',
 'CB',
 'PLD',
 'LRCX',
 'UPS',
 'MO',
 'PYPL',
 'NKE',
 'AMT',
 'EQIX',
 'GEV',
 'SO',
 'TT',
 'ICE',
 'APH',
 'ELV',
 'CMG',
 'KLAC',
 'INTC',
 'PH',
 'CME',
 'CRWD',
 'SHW',
 'DUK',
 'CDNS',
 'MDLZ',
 'ZTS',
 'PNC',
 'USB

In [30]:
import yfinance as yf
import datetime as dt
from datetime import timedelta
import pandas_ta as ta
import pandas as pd

In [95]:
years = .5
end = dt.datetime.now()
start = end - timedelta(days=365*years)

In [99]:
# get data
data = yf.download('TSLA', start=start, end=end)
data.columns = data.columns.str.lower()
data.rename_axis('date', inplace=True)
#calculate RSI column
data['ema_9'] = ta.ema(data['close'], length=9)
data['ema_30'] = ta.ema(data['close'], length=30)

adx = ta.adx(data['high'], data['low'],data['close'])           
adx.drop(columns=['DMP_14','DMN_14'], inplace=True)
adx.reset_index(inplace=True)
adx.columns = adx.columns.str.lower()

data.reset_index(inplace=True)
        
data = data.merge(adx, how='inner')

data.drop(columns=['adj close'], inplace=True)
data.dropna(inplace=True)

[*********************100%***********************]  1 of 1 completed


In [100]:
data

,date,open,high,low,close,volume,ema_9,ema_30,adx_14
29,2024-07-30,232.250000,232.410004,220.000000,222.619995,100560300,230.850030,225.336000,26.309992
30,2024-07-31,227.899994,234.679993,226.789993,232.070007,67497000,231.094025,225.770452,23.990577
31,2024-08-01,227.690002,231.869995,214.330002,216.860001,83861900,228.247220,225.195584,23.232422
32,2024-08-02,214.880005,216.130005,205.779999,207.669998,82880100,224.131776,224.064901,23.285589
33,2024-08-05,185.220001,203.880005,182.000000,198.880005,100308800,219.081422,222.440069,24.840203
...,...,...,...,...,...,...,...,...,...
122,2024-12-10,392.679993,409.730011,390.850006,400.989990,97563600,371.898804,330.895882,46.063567
123,2024-12-11,409.700012,424.880005,402.380005,424.769989,104287600,382.473041,336.952276,47.704553
124,2024-12-12,424.839996,429.299988,415.000000,418.100006,87752200,389.598434,342.187613,49.306868
125,2024-12-13,420.000000,436.299988,415.709991,436.230011,89000200,398.924749,348.254865,50.916589


In [102]:
message = ''
if data['ema_9'].iloc[-1] > data['ema_30'].iloc[-1]:
    message += 'Stock has momentum'

if data['adx_14'].iloc[-1]>25:
    message += ' and is trending'
    
print(message)


Stock has momentum and is trending
